In [80]:
import pandas as pd
import numpy as np
from we_eat.ALS_recommender import ALSRecommender
from survey_results import survey
import pickle
from pymongo import MongoClient

In [11]:
we_eat_client = MongoClient()

In [12]:
we_eat_database = we_eat_client['we_eat']
survey_collection = we_eat_database['surveys']

In [6]:
item_factors_df = pd.read_pickle('webapp/data/item_factors_df.pkl')

In [7]:
with open('webapp/data/inv_alias_dict.pickle', 'rb') as f:
    inv_alias_dict = pickle.load(f)

In [28]:
user1 = survey_collection.find_one({'user': 'gabe'})

In [29]:
user1

{'_id': ObjectId('5c0fee7691d56f3b9ac1e345'),
 'survey': {'green-leaf-vietnamese-restaurant-seattle': 8,
  'il-corvo-pasta-seattle': 9,
  'mee-sum-pastry-seattle': 0},
 'user': 'gabe'}

In [45]:
user2 = survey_collection.find_one({'user': 'jack'})

In [50]:
user2_df = recommender.user_preds_from_survey(user2)
user2_df

,amazon-go-seattle-5,the-pink-door-seattle-4,nirmals-seattle,biscuit-bitch-seattle-5,damn-the-weather-seattle,bad-bishop-seattle,pike-place-chowder-seattle,tsukushinbo-seattle,taylor-shellfish-oyster-bar-seattle-2,elliotts-oyster-house-seattle-2,...,mediterranean-cuisine-seattle,ricenroll-seattle-2,union-deli-la-puget-sound-plaza-building-seattle,measure-seattle,subway-seattle-3,u-bank-cafe-seattle,subway-seattle-31,subway-seattle-66,organic-to-go-seattle-4,chez-dave-seattle-2
jack,3.360238,2.930531,2.953157,3.842759,4.538444,2.334373,1.930434,1.12574,3.879147,2.841792,...,2.179335,1.104366,-0.865688,2.236233,3.464141,1.55824,1.787557,4.806378,2.323651,3.018861


In [81]:
recommender = ALSRecommender(item_factors_df, inv_alias_dict)

In [88]:
gabe = recommender.user_preds_from_survey(user1)

In [102]:
gabejack = pd.concat([gabe, user2_df], axis=1, sort=False)
gabejack

,amazon-go-seattle-5,the-pink-door-seattle-4,nirmals-seattle,biscuit-bitch-seattle-5,damn-the-weather-seattle,bad-bishop-seattle,pike-place-chowder-seattle,tsukushinbo-seattle,taylor-shellfish-oyster-bar-seattle-2,elliotts-oyster-house-seattle-2,...,mediterranean-cuisine-seattle,ricenroll-seattle-2,union-deli-la-puget-sound-plaza-building-seattle,measure-seattle,subway-seattle-3,u-bank-cafe-seattle,subway-seattle-31,subway-seattle-66,organic-to-go-seattle-4,chez-dave-seattle-2
gabe,4.467413,1.929282,2.93576,1.890571,1.690533,2.315328,2.863935,5.852221,2.394073,3.751703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jack,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.179335,1.104366,-0.865688,2.236233,3.464141,1.55824,1.787557,4.806378,2.323651,3.018861


In [110]:
list(gabejack.index)

['gabe', 'jack']

In [84]:
def recommend(user1, user2):
    """Return a recommendation, given two usernames."""
    #data = request.get_json()
    user1 = survey_collection.find_one({'user': user1})
    user2 = survey_collection.find_one({'user': user2})
    recommender = ALSRecommender(item_factors_df, inv_alias_dict)
    user1_df = recommender.user_preds_from_survey(user1)
    user2_df = recommender.user_preds_from_survey(user2)
    preds_df = recommender.compile_df(user1_df, user2_df)
    top_rec = recommender.top_rec(user1, user2, preds_df)
    random_rec = recommender.get_a_rec(user1, user2, preds_df)
    return random_rec.index[0], top_rec.index[0]

In [94]:
recommend('gabe', 'jack')

KeyError: "None of [{'_id': ObjectId('5c0fee7691d56f3b9ac1e345'), 'user': 'gabe', 'survey': {'green-leaf-vietnamese-restaurant-seattle': 8, 'il-corvo-pasta-seattle': 9, 'mee-sum-pastry-seattle': 0}}] are in the [index]"

In [96]:
recommender.top_recs('gabe', 'jack')

AttributeError: 'ALSRecommender' object has no attribute 'top_recs'